In [ ]:
import pymysql
from datetime import datetime
from oBIX import Client
import pandas as pd
import psycopg2
import time
from threading import Thread
import numpy as np
import schedule
import math


In [ ]:
datetime.now()

In [ ]:
global tmp
tmp=pd.read_excel(io='./tmp_修改了设备编号.xls',sheet_name=0)
# 使用前向填充方法填充 NaN 值（向前填充）
tmp = tmp.fillna(method='ffill', axis=0)
tmp=np.array(tmp).tolist()

global dl
dl=[]

global time_count
time_count=0

device_list=['温度传感器','压力传感器','阀门','地源泵','用户泵','锅炉循环泵','氢燃料电池预热循环泵','蓄水箱循环泵','蓄水箱循环泵','补水泵','主循环泵','主循环泵','进地热井','蓄水箱液位','补水箱液位','蓄能水箱压力监测',
             '出蓄水箱总管温度','出蓄水箱总管压力','能源站回水管除污器前压力监测','能源站回水管除污器后压力监测','接DK-1及DK-2地块供水管压力监测','接DK-1及DK-2地块供水管温度监测','锅炉回水总管温度监测','锅炉回水总管压力监测','旁通管压差监测',
             '补水泵阀门','电表','出地热井水管总管水管冷热量监测','能源站供水管冷热量监测','接DK-1及DK-2地块供水管冷热量监测','氢燃料电池余热冬季蓄热时出水管冷热量监测','地源热泵夏季夜间往蓄水罐蓄冷时出水管冷热量监测',
             '电锅炉冬季蓄热时出水管冷热量监测','进地热井水管总管流量监测','地源热泵夏季夜间往蓄水罐蓄冷时供水管冷热量监测','地源热泵蒸发侧流量','地源热泵地源侧流量','水表','地源热泵','水泵','锅炉','平台控制相关点位',
             '能源中心板换','消防水池板换','运动员村','氢燃料电池','水箱温度','水箱液位']

In [ ]:
# 连接数据库，并往数据库里建表（最后不一定需要）
# def create_mysql_table_obix(t_n): 
#     mysql_connection = pymysql.connect(host = '123.249.70.226',port=7047,user='root',password='ems123@4',database='ems_test')
    

#     sql = f"""CREATE TABLE IF NOT EXISTS {t_n} (
#         gid INT AUTO_INCREMENT PRIMARY KEY COMMENT '检索主键',
#         create_time TIMESTAMP NOT NULL COMMENT '数据插入的时间',
#         item_addr VARCHAR(50) NOT NULL COMMENT '点位数据地址',
#         item_name VARCHAR(100) COMMENT '点位数据名称或备注',
#         item_unit VARCHAR(100) COMMENT '点位数据单位或范围',
#         item_val VARCHAR(50)  COMMENT '数据值'
#     ) COMMENT'存储obix数据,一分钟存储一次'  ;    
#     """
#     cursor = mysql_connection.cursor()
#     cursor.execute(sql)
#     mysql_connection.commit()
#     mysql_connection.close()

In [ ]:

# 给定客户端和地址，就可以读地址的数据  注意插入的数值不能为nan
#读取到的有些值为bool类型，要转为float
def read_obix(client, point_path):
    point_value = client.read_point_value(point_path)
    if isinstance(point_value, float) and not math.isnan(point_value):
        point_value = round(point_value, 10)  # 保留十位小数
    elif isinstance(point_value, bool):
        point_value = float(point_value)  # 将布尔值转换为相应的数值类型，比如 1 或 0
    # else:
    #     point_value=0
    
    return point_value


In [ ]:
def process_device_string(device_string, device_list):
    #清除换行符
    device_string = device_string.replace('\n', '') 
    device_name = None
    device_id = None
    for device in device_list:
        if device in device_string:
            device_name = device
            device_id = device_string.replace(device, '')
            
            if not device_id:
                device_id = '1'
            break  # 找到匹配的设备后跳出循环
    return device_id,device_name  # 如果未找到匹配项，则返回 None

# #设备列表
# devices = ['温度传感器', '压力传感器', '阀门', '地源泵', '用户泵', '锅炉循环泵', '氢燃料电池预热循环泵', '蓄水箱循环泵', '蓄水箱循环泵', '补水泵','旁通管压差监测']

# # 测试示例
# device_string_1 = '旁通管压差监测\n△P'
# device_name_1, device_id_1 = process_device_string(device_string_1, devices)
# print(f"Device Name: {device_name_1}, Device ID: {device_id_1}")

# device_string_2 = '用户泵'
# device_name_2, device_id_2 = process_device_string(device_string_2, devices)
# print(f"Device Name: {device_name_2}, Device ID: {device_id_2}")



In [ ]:
# ##测试能否读取数值
# i = 0
# while True:
#     i=i+1
#     print('==================================================================================================')
#     point=tmp.pop(0)
#     print("point",point)
#     device = point[0]
#     device=str(device)     #试一下强制转换
#     print("device:",device)
#     print("device_type:",type(device))
#     addr=point[1].split('points')[1].replace('//','/')
#     print("addr:",addr)
#     name=str(point[4]) if len(str(point[4]))<100 else str(point[4])[:100]
#     print("name:",name)
#     unit=str(point[5]) if len(str(point[5]))<100 else str(point[5])[:100]
#     print("unit:",unit)
#     point_path=point[1].split('obix')[1].replace('//','/')
#     print("point_path:",point_path)
#     client = Client("123.249.70.226", "obix", "Honeywell2022", port = 7006, https=False)
#     load_value=read_obix(client,point_path)   #这样的time不一致
#     print("load_value:",load_value)
#     print("load_value_type:",type(load_value))
    
    

#     device_name_2, device_id_2 = process_device_string(device, device_list)
#     print(f"Device Name: {device_name_2}, Device ID: {device_id_2},第{i}行")

In [ ]:
# 要处理这个表，根究表的信息，给出[load_time,system_id,system_name,device_id,device_name,position_id,position_name,load_value]
# obix_thread函数执行一遍，就得到tmp表的每行解析出来的reate_time,addr,name,unit,value
def obix_thread():
    #client = Client("192.168.6.60", "obix", "Honeywell2022", port = 8082, https=False)
    client = Client("123.249.70.226", "obix", "Honeywell2022", port = 7006, https=False)
    
    i=0
    #这个循环会把都tmp表的每一行都遍历一边
    while(len(tmp)>0): 
        i=i+1
        point=tmp.pop(0)
        load_time=datetime.now().strftime("%Y-%m-%d %H:%M")
        system_id= '1'       # 1
        system_name= '暖通系统'     #暖通系统
        device_string = point[0]
        device_id,device_name=process_device_string(device_string,device_list)#   
        position_id=point[1].split('points')[1].replace('//','/')
        position_name=str(point[4]) if len(str(point[4]))<100 else str(point[4])[:100]
        # unit=str(point[5]) if len(str(point[5]))<100 else str(point[5])[:100]
        load_value=read_obix(client,point[1].split('obix')[1].replace('//','/'))
        print("load_value:",load_value,type(load_value),i)
        dl.append([load_time,system_id,system_name,device_id,device_name,position_id,position_name,load_value])
    print("dl的长度为:",len(dl))
    print(dl)

In [ ]:
obix_thread()

In [ ]:
print("准备开始：将数据数据传入数据库之中") 
# 连接数据库，并写值进去
mysql_connection = pymysql.connect(host = '123.249.70.226',port=7047,user='root',password='ems123@4',database='ems_backup')
for d in dl:
    if d[7] !=None:
        cursor = mysql_connection.cursor()
        #sql='INSERT INTO  ems_els_history (load_time,system_id,system_name,device_id,device_name,position_id,position_name,load_value)'+f"VALUES('{d[0]}','{d[1]}','{d[2]}','{d[3]}','{d[4]}','{d[5]}','{d[6]}','{d[7]}');" #当成字符串了
        #sql='INSERT INTO  ems_els_history (load_time,system_id,system_name,device_id,device_name,position_id,position_name,load_value) VALUES(d[0],d[1],d[2],d[3],d[4],d[5],d[6],d[7])'
        # sql='INSERT INTO  ems_els_history (load_time,system_id,system_name,device_id,device_name,position_id,position_name,load_value) VALUES(%s,%s,%s,%s,%s,%s,%s,%s)'
        # cursor.execute(sql, (d[0],d[1],d[2],d[3],d[4],d[5],d[6],d[7]))

        sql = 'INSERT INTO ems_els_history (load_time, system_id, system_name, device_id, device_name, position_id, position_name, load_value) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)'
        try:
            cursor.execute(sql, (d[0], d[1], d[2], d[3], d[4], d[5], d[6], d[7]))
        except Exception as e:
            print(f"Unable to insert data: {d}, Error: {e}")   #有些写不进去

mysql_connection.commit()
mysql_connection.close()
print("已经将数据数据传入数据库之中")  # 4203  这一步没过来

In [ ]:
def duqu1():
    # time_s=time.time()
    global dl
    global tmp
    print(datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

    # t_l=[]
    # for i in range(4):
    #     obix_t=Thread(target=obix_thread,args=())
    #     t_l.append(obix_t)
    # # post_t=Thread(target=postgre_write_thread,args=())
    # for t in t_l:
    #     t.start()
    # for t in t_l:
    #     t.join()

    # 连接数据库，并写值进去
    mysql_connection = pymysql.connect(host = '123.249.70.226',port=7047,user='root',password='ems123@4',database='ems_backup')
    for d in dl:
        if d[7] !=None:
            sql='INSERT INTO  ems_els_history (load_time,system_id,system_name,device_id,device_name,position_id,position_name,load_value)'+f"VALUES('{d[0]}','{d[1]}','{d[2]}','{d[3]}','{d[4]}','{d[5]}','{d[6]}','{d[7]}');"
            cursor = mysql_connection.cursor()
            cursor.execute(sql)
    mysql_connection.commit()
    mysql_connection.close()


    dl=[]
    tmp=pd.read_excel(io='tmp_lxy_修改了设备编号.xls',sheet_name=0)
    tmp=np.array(tmp).tolist()

    # time_e=time.time()
    # print('postgre'+str(time_s-time_e))

# obix地址编码 ,并生成新的processed_tmp_lxy


In [39]:
import pandas as pd 
#########################################################################################################################
#功能：对原表处理，产出processed_obix表格
#########################################################################################################################
device_list=['温度传感器','压力传感器','阀门','地源泵','用户泵','锅炉循环泵','氢燃料电池预热循环泵','蓄水箱循环泵','蓄水箱循环泵','补水泵','主循环泵','主循环泵','进地热井','蓄水箱液位','补水箱液位','蓄能水箱压力监测',
             '出蓄水箱总管温度','出蓄水箱总管压力','能源站回水管除污器前压力监测','能源站回水管除污器后压力监测','接DK-1及DK-2地块供水管压力监测','接DK-1及DK-2地块供水管温度监测','锅炉回水总管温度监测','锅炉回水总管压力监测','旁通管压差监测',
             '补水泵阀门','电表','出地热井水管总管水管冷热量监测','能源站供水管冷热量监测','接DK-1及DK-2地块供水管冷热量监测','氢燃料电池余热冬季蓄热时出水管冷热量监测','地源热泵夏季夜间往蓄水罐蓄冷时出水管冷热量监测',
             '电锅炉冬季蓄热时出水管冷热量监测','进地热井水管总管流量监测','地源热泵夏季夜间往蓄水罐蓄冷时供水管冷热量监测','地源热泵蒸发侧流量','地源热泵地源侧流量','水表','地源热泵','水泵','锅炉','平台控制相关点位',
             '能源中心板换','消防水池板换','运动员村','氢燃料电池','水箱温度','水箱液位']

In [44]:
df = pd.read_excel("./tmp_修改了设备编号.xls")
df = df.fillna(method='ffill', axis=0)


In [45]:
df['点位地址'] = df['点位地址'].apply(lambda x: x.split('points')[1].replace('//', '/'))


In [46]:
def process_device_string(device_string, device_list):
    #清除换行符
    device_string = device_string.replace('\n', '') 
    device_name = None
    device_id = None
    for device in device_list:   #遍历表里所有设备
        if device in device_string:
            device_name = device
            device_id = device_string.replace(device, '')   #修改这一段
            #知道了 device_id（T_2） 和device_name（温度传感器）  直接读取一个表格就行了  返回合适的 device_id（01） 和device_name（温度传感器）
            
            if not device_id:
                device_id = '01'
            break  # 找到匹配的设备后跳出循环
    return device_id,device_name # 如果未找到匹配项，则返回 None

In [47]:
# df['device_id'],df['device_name'] = df['点位地址'].apply(lambda x: process_device_string(x, device_list))
# process_device_string 返回两个值
df[['device_id', 'device_name']] = df['设备编号'].apply(lambda x: pd.Series(process_device_string(x, device_list)))
df

,设备编号,点位地址,点位类型,读写权限,注释,Unnamed: 5,Unnamed: 6,device_id,device_name
0,温度传感器T_1,/CH_1/T_1/,Real,True,温度,单位：℃,NaN,T_1,温度传感器
1,温度传感器T_2,/CH_1/T_2/,Real,True,温度,单位：℃,NaN,T_2,温度传感器
2,压力传感器P_1,/CH_1/P_1/,Real,True,压力,单位：bar,NaN,P_1,压力传感器
3,压力传感器P_2,/CH_1/P_2/,Real,True,压力,单位：bar,NaN,P_2,压力传感器
4,温度传感器T_3,/CH_1/T_3/,Real,True,温度,单位：℃,NaN,T_3,温度传感器
...,...,...,...,...,...,...,...,...,...
664,水箱温度,/SX_T/T21,Real,True,温度,单位：℃,底部,01,水箱温度
665,水箱温度,/SX_T/T22,Real,True,温度,单位：℃,底部,01,水箱温度
666,水箱温度,/SX_T/T23,Real,True,温度,单位：℃,底部,01,水箱温度
667,水箱温度,/SX_T/T24,Real,True,温度,单位：℃,顶部,01,水箱温度


In [48]:
# 对 device_name 和 device_id 进行分组
grouped = df.groupby(['device_name', 'device_id'])  #将相同 device_name 和 device_id 的行放在同一个组中

# 重新编号
new_device_ids = {}
new_device_id_count = {}

for (device_name, device_id), group in grouped:  #温度传感器  1     温度传感器  2      压力传感器  1    压力传感器  2     压力传感器  3      温度传感器  3   （这是迭代对象）
    ######################################################################################################################################
    #功能：如果新进来一个设备，那么就为他构建两个字典
    if device_name not in new_device_ids:
        new_device_ids[device_name] = {}       # 为新的device_name建立字典            字典的键：DC/DC设备   字典的值 ：一个字典
        new_device_id_count[device_name] = 1   # 另一个字典 对id重新编号，过度用的  保留每个device的id的最后值
    ######################################################################################################################################

    #如果进来的设备的字典已经建立 
    new_device_ids[device_name][device_id] = f"{new_device_id_count[device_name]:02d}"    #两位数字格式  原来的id是1，现在变为01
    new_device_id_count[device_name] += 1

# 创建新的列 device_id_00
# 对 DataFrame 的每一行执行自定义的函数。在这个例子中，lambda 函数接收每一行，并根据 device_name 和 device_id 查找新的设备编号，然后将结果赋给名为 device_id_00 的新列。
df['device_id_00'] = df.apply(lambda row: new_device_ids[row['device_name']][row['device_id']], axis=1)

In [49]:
# 生成从1开始的新列
df['position_id'] = range(1, len(df) + 1)
df['position_id'] = df['position_id'].apply(lambda x: f"{x:06}")
df.rename(columns={'注释': 'position_name'}, inplace=True) 
df

,设备编号,点位地址,点位类型,读写权限,position_name,Unnamed: 5,Unnamed: 6,device_id,device_name,device_id_00,position_id
0,温度传感器T_1,/CH_1/T_1/,Real,True,温度,单位：℃,NaN,T_1,温度传感器,01,000001
1,温度传感器T_2,/CH_1/T_2/,Real,True,温度,单位：℃,NaN,T_2,温度传感器,02,000002
2,压力传感器P_1,/CH_1/P_1/,Real,True,压力,单位：bar,NaN,P_1,压力传感器,01,000003
3,压力传感器P_2,/CH_1/P_2/,Real,True,压力,单位：bar,NaN,P_2,压力传感器,02,000004
4,温度传感器T_3,/CH_1/T_3/,Real,True,温度,单位：℃,NaN,T_3,温度传感器,03,000005
...,...,...,...,...,...,...,...,...,...,...,...
664,水箱温度,/SX_T/T21,Real,True,温度,单位：℃,底部,01,水箱温度,01,000665
665,水箱温度,/SX_T/T22,Real,True,温度,单位：℃,底部,01,水箱温度,01,000666
666,水箱温度,/SX_T/T23,Real,True,温度,单位：℃,底部,01,水箱温度,01,000667
667,水箱温度,/SX_T/T24,Real,True,温度,单位：℃,顶部,01,水箱温度,01,000668


In [50]:

df['position_id_database'] = df.apply(lambda row: '10'+row['device_id_00'] + row['position_id'], axis=1)

# 打印结果

In [51]:
df

,设备编号,点位地址,点位类型,读写权限,position_name,Unnamed: 5,Unnamed: 6,device_id,device_name,device_id_00,position_id,position_id_database
0,温度传感器T_1,/CH_1/T_1/,Real,True,温度,单位：℃,NaN,T_1,温度传感器,01,000001,1001000001
1,温度传感器T_2,/CH_1/T_2/,Real,True,温度,单位：℃,NaN,T_2,温度传感器,02,000002,1002000002
2,压力传感器P_1,/CH_1/P_1/,Real,True,压力,单位：bar,NaN,P_1,压力传感器,01,000003,1001000003
3,压力传感器P_2,/CH_1/P_2/,Real,True,压力,单位：bar,NaN,P_2,压力传感器,02,000004,1002000004
4,温度传感器T_3,/CH_1/T_3/,Real,True,温度,单位：℃,NaN,T_3,温度传感器,03,000005,1003000005
...,...,...,...,...,...,...,...,...,...,...,...,...
664,水箱温度,/SX_T/T21,Real,True,温度,单位：℃,底部,01,水箱温度,01,000665,1001000665
665,水箱温度,/SX_T/T22,Real,True,温度,单位：℃,底部,01,水箱温度,01,000666,1001000666
666,水箱温度,/SX_T/T23,Real,True,温度,单位：℃,底部,01,水箱温度,01,000667,1001000667
667,水箱温度,/SX_T/T24,Real,True,温度,单位：℃,顶部,01,水箱温度,01,000668,1001000668


In [53]:
# 输出为新的文件
df.to_excel('processed_obix_database.xlsx',index=False)